# Начало анализа

In [92]:
import pandas as pd
# from currency_converter import CurrencyConverter



df = pd.read_csv('_data.csv', index_col=0)
# df.columns
# df['ID  объявления'].value_counts()
# df.head(30)
df.info()
# df['Количество комнат'].value_counts()
# df.loc[:15, 'Площадь, м2']

<class 'pandas.core.frame.DataFrame'>
Index: 23368 entries, 0 to 23367
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID  объявления            23368 non-null  int64  
 1   Количество комнат         22327 non-null  object 
 2   Тип                       23368 non-null  object 
 3   Метро                     22053 non-null  object 
 4   Адрес                     23368 non-null  object 
 5   Площадь, м2               23368 non-null  object 
 6   Дом                       23368 non-null  object 
 7   Парковка                  9951 non-null   object 
 8   Цена                      23368 non-null  object 
 9   Телефоны                  23368 non-null  object 
 10  Описание                  23368 non-null  object 
 11  Ремонт                    20613 non-null  object 
 12  Площадь комнат, м2        14458 non-null  object 
 13  Балкон                    15390 non-null  object 
 14  Окна       

### Достаем данные Москвы

In [93]:
# Город
df['city'] = [row.split(',')[0] if isinstance(row, str) else row for row in df['Адрес']]
df['city'].value_counts()
# df_msk = df[df['Адрес'].str.contains("Москва")].reset_index(drop = True)
df = df[df['city']=='Москва']
df.shape

(19737, 25)

## Работаем с пропусками

In [94]:
# Cмотрим NaN значения
df.isna().sum()

ID  объявления                  0
Количество комнат             535
Тип                             0
Метро                         346
Адрес                           0
Площадь, м2                     0
Дом                             0
Парковка                    11174
Цена                            0
Телефоны                        0
Описание                        0
Ремонт                       2463
Площадь комнат, м2           7228
Балкон                       6630
Окна                         5150
Санузел                      2041
Можно с детьми/животными     4915
Дополнительно                 272
Название ЖК                 15281
Серия дома                  17646
Высота потолков, м           9202
Лифт                         4192
Мусоропровод                 8007
Ссылка на объявление            0
city                            0
dtype: int64

### Комнаты

In [95]:
# Получаем количество комнат
df.loc[:,'Число комнат'] = [row.split(',')[0] if isinstance(row, str) else row for row in df['Количество комнат']]
df['Число комнат'].unique()
df['Число комнат'].value_counts()
df['Число комнат'].isna().sum()

# Анализ выборки NaN в графе комнат показал, что это квартиры-студии, заменим NaN на 0
df.loc[df['Число комнат'].isna()]['Описание'].head()
df['Число комнат'].fillna(0, inplace=True)

### Метро

In [96]:
# Колонка метро
df['Метро2'] = [row.split(' (')[0] if isinstance(row, str) else row for row in df['Метро']]
df['Метро2'] = [row.split('. ')[1] if isinstance(row, str) else row for row in df['Метро2']]

# df['Метро2'].fillna('Неизвестно', inplace=True)
df['Метро2'].isna().sum()

df['maybe_subway'] = df['Описание'].str.contains('метро', regex=False)
df['maybe_subway']

# нельзя из Описания вытащить новые станции метро 
df.loc[(df['Метро2'].isna()) & (df['maybe_subway']==True)]['Метро2'].count()
df.loc[df['Метро'].isna()]['Дополнительно']

189      Мебель в комнатах, Мебель на кухне, Стиральная...
233      Мебель в комнатах, Мебель на кухне, Душевая ка...
244      Мебель в комнатах, Мебель на кухне, Ванна, Сти...
267      Мебель в комнатах, Мебель на кухне, Ванна, Сти...
745                                               Интернет
                               ...                        
21970    Мебель в комнатах, Мебель на кухне, Стиральная...
21971    Мебель в комнатах, Мебель на кухне, Стиральная...
22160    Мебель в комнатах, Мебель на кухне, Стиральная...
23273            Стиральная машина, Телевизор, Холодильник
23317                       Стиральная машина, Холодильник
Name: Дополнительно, Length: 346, dtype: object

### Адрес

In [97]:
# Создаем переменную "за мкадом"
df['Адрес'].unique()
df['За_МКАД'] = df['Адрес'].apply(lambda x: 1 if 'мкад' in str(x).lower() else 0)
df['За_МКАД'].value_counts()

За_МКАД
0    18187
1     1550
Name: count, dtype: int64

In [98]:
# Удаляем концы адреса по типу "ш. Варшавское (4 км до МКАД)"
df['Новый Адрес'] = [row.split('ш. ')[0] for row in df['Адрес']]
# df_msk = df_msk[df_msk['Адрес'] != 'Москва, Жилой Поколение кв-л'].reset_index(drop = True)

In [99]:
# Привязать адрес к районам / спарсить "садовое" из описания

# TBD!


### Площадь

In [100]:
df['Площадь, м2'].unique()
# Получим общую площадь квартиры
df['total_square'] = [row.split('.')[0] if isinstance(row, str) else row for row in df['Площадь, м2']]
df['total_square'].astype(int)
# df['total_square'].unique()

0        200
1        198
2        200
3        170
4         58
        ... 
23363     35
23364     38
23365     43
23366     52
23367     90
Name: total_square, Length: 19737, dtype: int64

In [101]:
# получим площадь кухни
df['Площадь, м2'].unique()
df['kitchen_square'] = [row.split('/')[-1] if isinstance(row, str) else row for row in df['Площадь, м2']]
# df_msk[['Площадь, м2', 'kitchen_square']]
# df_msk['kitchen_square'].isna().sum()

### Животные и дети

In [102]:
df['Можно с детьми/животными'].unique()
df['ok_pets'] = df['Можно с детьми/животными'].apply(lambda x: 1 if 'животными' in str(x).lower() else 0)
df['ok_kids'] = df['Можно с детьми/животными'].apply(lambda x: 1 if 'детьми' in str(x).lower() else 0)
# df_msk['ok_kids'].sum()
# df_msk['ok_pets'].sum()

### Балкон

In [103]:
df['Балкон'].unique()

df['Число балконов'] = df['Балкон'].str.extract(r'Балкон \((\d+)\)').astype(float)
df['Число лоджий'] = df['Балкон'].str.extract(r'Лоджия \((\d+)\)').astype(float)

df['Число балконов'].isna().sum()
df['Число балконов'] = df['Число балконов'].fillna(df['Число балконов'].mean())
# df_msk['Число балконов'].isna().sum()
df['Число лоджий'].isna().sum()
df['Число лоджий'] = df['Число лоджий'].fillna(df['Число лоджий'].mean())

### Санузел

In [104]:
df['Санузел']
df['Число совмещенных су'] = df['Санузел'].str.extract(r'Совмещенный \((\d+)\)').astype(float)
df['Число раздельных су'] = df['Санузел'].str.extract(r'Раздельный \((\d+)\)').astype(float)

df['Число совмещенных су'] = df['Число совмещенных су'].fillna(df['Число совмещенных су'].mean())
df['Число раздельных су'] = df['Число раздельных су'].fillna(df['Число раздельных су'].mean())
# df_msk['Число совмещенных су'].isna().sum()

### Мебель и кондиционер

In [105]:
df['Дополнительно']
# Мебель в комнатах
# Кондиционер
df['Есть мебель'] = df['Дополнительно'].str.extract('(ебель в комнатах)',).astype(str)
df['Есть кондиционер'] = df['Дополнительно'].str.extract('(ондиционер)',).astype(str)

df['Есть мебель'] = [1 if 'ебель в комнатах' in row else 0 for row in df['Есть мебель']]
df['Есть кондиционер'] = [1 if 'ондиционер' in row else 0 for row in df['Есть кондиционер']]


### Парковка

In [106]:
df['Парковка'] = df['Парковка'].replace(to_replace ='открытая', value = 'бесплатная во дворе')
df['Парковка'] = df['Парковка'].replace(to_replace ='наземная', value = 'платная во дворе')
df['Парковка'] = df['Парковка'].replace(to_replace ='многоуровневая', value = 'платная во дворе')
df['Парковка'] = df['Парковка'].replace(to_replace ='подземная', value = 'платная во дворе')
df['Парковка'] = df['Парковка'].replace(to_replace ='на крыше', value = 'платная во дворе')
df['Парковка'] = df['Парковка'].fillna('бесплатная во дворе')
df['Parking'] = df['Парковка']
df = df.drop(['Парковка'], axis = 1)

### Дом (тип, этажность, этаж)

In [107]:
df['house_type'] = [row.split(', ')[-1] if len(row.split(', ')[-1]) > 6 else 'Неизвестно' for row in df['Дом']]
df['floors_amount'] = [int(row.split('/')[1].split(',')[0]) for row in df['Дом']]
df['flat_floor'] = [int(row.split('/')[0]) for row in df['Дом']]
df = df.drop(['Дом'], axis = 1)

### Цена


In [108]:
c = CurrencyConverter()
def sale(text):
    result = None
    if text.split(' ')[1] == 'руб./':
        result = float(text.split(' ')[0])
    elif text.split(' ')[1] == '$,':
        result = c.convert(float(text.split(' ')[0]), 'USD', 'RUB')
    elif text.split(' ')[1] == '€,':
        result = c.convert(float(text.split(' ')[0]), 'EUR', 'RUB')
    return result
text = df['Цена']
text = text.apply(sale)
df['Price'] = text

NameError: name 'CurrencyConverter' is not defined

In [109]:
# влючена ли комуналка
df['Utilities'] = df['Цена'].str.contains('Коммунальные услуги включены')
df['Rent_length'] = df['Цена'].str.contains('Срок аренды - Длительный')
df['Prepay'] = df['Цена'].str.contains('Предоплата')

In [110]:
# лифты

df['Лифт'] = [
    row if row == row else
    ['Пасс (0), Груз (0)' if 5 >= df['floors_amount'].iloc[i]  else
     'Пасс (1), Груз (0)' if 5 <= df['floors_amount'].iloc[i] <= 9  else
     'Пасс (1), Груз (1)' if 10 <= df['floors_amount'].iloc[i] <= 19 else
     'Пасс (2), Груз (1)'][0]
    for i, row in enumerate(df['Лифт'])
]
df['Pass_elevator'] = [row.split(' ')[1] if (row.split(' ')[0]) == 'Пасс' else '(0)' for row in df['Лифт']]
df['Cargo_elevator'] = [row.split(' ')[-1] if (row.split(' ')[-2]) == 'Груз' else '(0)' for row in df['Лифт']]

df['Pass_elevator'] = [row.split('(')[1] for row in df['Pass_elevator']]
df['Pass_elevator'] = [int(row.split(')')[0]) for row in df['Pass_elevator']]
df['Cargo_elevator'] = [row.split('(')[1] for row in df['Cargo_elevator']]
df['Cargo_elevator'] = [int(row.split(')')[0]) for row in df['Cargo_elevator']]
df = df.drop(['Лифт'], axis = 1)

### Окна, потолки, мусоропровод, ремонт

In [111]:
df['windows'] = df["Окна"].fillna('Во двор')

In [112]:
df['ceiling_height'] = df['Высота потолков, м'].fillna(round(df['Высота потолков, м'].mean(), 2))

In [113]:
df['Garbage_disposal'] = df['Мусоропровод'].fillna('Нет')

In [114]:
df['maintenance'] = df['Ремонт'].fillna('Без ремонта')

### Удаление старых / лишних колонок

In [119]:
df.columns

df.drop(['ID  объявления', 'Площадь, м2', 'Количество комнат', 'Тип', 'Адрес', 'Цена', 'Телефоны', 'Описание',
'Ремонт', 'Площадь комнат, м2', 'Балкон', 'Окна', 'Санузел', 'Можно с детьми/животными', 'Дополнительно', 'Название ЖК', 'Серия дома', 'Высота потолков, м', 'Мусоропровод',
'Ссылка на объявление', 'city', 'maybe_subway'], axis=1, inplace=True)

# df.drop(['ID  объявления'], inplace=True)

In [120]:
df.isna().sum()

Метро                   346
Число комнат              0
Метро2                  346
За_МКАД                   0
Новый Адрес               0
total_square              0
kitchen_square            0
ok_pets                   0
ok_kids                   0
Число балконов            0
Число лоджий              0
Число совмещенных су      0
Число раздельных су       0
Есть мебель               0
Есть кондиционер          0
Parking                   0
house_type                0
floors_amount             0
flat_floor                0
Utilities                 0
Rent_length               0
Prepay                    0
Pass_elevator             0
Cargo_elevator            0
windows                   0
ceiling_height            0
Garbage_disposal          0
maintenance               0
dtype: int64

In [121]:
df.dtypes

Метро                    object
Число комнат             object
Метро2                   object
За_МКАД                   int64
Новый Адрес              object
total_square             object
kitchen_square           object
ok_pets                   int64
ok_kids                   int64
Число балконов          float64
Число лоджий            float64
Число совмещенных су    float64
Число раздельных су     float64
Есть мебель               int64
Есть кондиционер          int64
Parking                  object
house_type               object
floors_amount             int64
flat_floor                int64
Utilities                  bool
Rent_length                bool
Prepay                     bool
Pass_elevator             int64
Cargo_elevator            int64
windows                  object
ceiling_height          float64
Garbage_disposal         object
maintenance              object
dtype: object

In [122]:
df['Метро2']

0        Смоленская
1        Смоленская
2        Смоленская
3        Смоленская
4         Арбатская
            ...    
23363      Говорово
23364      Солнцево
23365      Солнцево
23366      Солнцево
23367      Говорово
Name: Метро2, Length: 19737, dtype: object